<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# Module 2: Preparing Data 

## Identifying Outliers

### Lesson Objectives

After completing this lesson, you should be able to:

- Compute the inverse of covariance matrix given of a dataset
-	Compute Mahalanobis Distance for all elements in a dataset
-	Remove outliers from a dataset


## Mahalanobis Distance 

-	Multi-dimensional generalization of measuring how many standard deviations a point is away from the mean
-	Measured along each Principal Component axis 
-	Unitless and scale-invariant 
-	Takes into account the correlations of the dataset
-	Used to detect outliers

In [1]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._

import org.apache.spark.sql.functions._
import  org.apache.spark.mllib.linalg.{Vector, Vectors}
import  org.apache.spark.ml.feature.StandardScaler
import  org.apache.spark.ml.feature.VectorAssembler 
import org.apache.spark.mllib.util.MLUtils


spark = org.apache.spark.sql.SparkSession@375f378b


org.apache.spark.sql.SparkSession@375f378b

In [2]:
// Continuing from Previous Example 

val  dfRandom = spark.range(0, 10).select("id").
 withColumn("uniform", rand(10L)).
 withColumn("normal1", randn(10L)).
 withColumn("normal2", randn(11L))

val  assembler = new VectorAssembler().
 setInputCols(Array("uniform","normal1","normal2")).
 setOutputCol("features")

val dfVec = MLUtils.convertVectorColumnsFromML(assembler.transform(dfRandom))

dfRandom = [id: bigint, uniform: double ... 2 more fields]
assembler = vecAssembler_02e163db81ca
dfVec = [id: bigint, uniform: double ... 3 more fields]


[id: bigint, uniform: double ... 3 more fields]

In [3]:
// Continuing from the Previous Example
dfVec.select("id","features").show()

// An Example with Outliers 
val dfOutlier = dfVec.select("id","features").unionAll(spark.createDataFrame(Seq((10,Vectors.dense(3, 3, 3)))))
dfOutlier.sort(dfOutlier("id").desc).show(5)

+---+--------------------+
| id|            features|
+---+--------------------+
|  0|[0.41371264720975...|
|  1|[0.73117192818966...|
|  2|[0.90317011551182...|
|  3|[0.09430205113458...|
|  4|[0.38340505276222...|
|  5|[0.55692461355235...|
|  6|[0.49774414066138...|
|  7|[0.20766661062014...|
|  8|[0.95719194065089...|
|  9|[0.74293954612044...|
+---+--------------------+

+---+--------------------+
| id|            features|
+---+--------------------+
| 10|       [3.0,3.0,3.0]|
|  9|[0.74293954612044...|
|  8|[0.95719194065089...|
|  7|[0.20766661062014...|
|  6|[0.49774414066138...|
+---+--------------------+
only showing top 5 rows



dfOutlier = [id: bigint, features: vector]


[id: bigint, features: vector]

In [4]:
// An Example with Outliers 

val scaler = new StandardScaler().
 setInputCol("features").setOutputCol("scaledFeat").
 setWithStd(true).setWithMean(true)

val scalerModel = scaler.fit(MLUtils.convertVectorColumnsToML(dfOutlier.select("id","features")))

val dfScaled = scalerModel.transform(MLUtils.convertVectorColumnsToML(dfOutlier)).select("id","scaledFeat")
dfScaled.sort(dfScaled("id").desc).show(3)

+---+--------------------+
| id|          scaledFeat|
+---+--------------------+
| 10|[2.83077635345669...|
|  9|[-0.0364814207361...|
|  8|[0.23569420610668...|
+---+--------------------+
only showing top 3 rows



scaler = stdScal_84c9dd6c5b36
scalerModel = stdScal_84c9dd6c5b36
dfScaled = [id: bigint, scaledFeat: vector]


[id: bigint, scaledFeat: vector]

In [6]:
import  org.apache.spark.mllib.stat.Statistics

import  breeze.linalg._

val  rddVec = MLUtils.convertVectorColumnsFromML(dfScaled.select("scaledFeat")).rdd.map(_(0).asInstanceOf[org.apache.spark.mllib.linalg.Vector])

val  colCov = Statistics.corr(rddVec)
val  invColCovB = inv(new DenseMatrix(3, 3, colCov.toArray))

rddVec = MapPartitionsRDD[63] at map at <console>:64
colCov = 
invColCovB = 


1.0                  0.48885755212288634  0.5651391787484287
0.48885755212288634  1.0                  0.26406564567048074
0.5651391787484287   0.26406564567048074  1.0
1.7965335033552017   -0.6558803463974368   -0.8420960015261783
-0.6558803463974368  1.314406990367565     0.023573949735077922
-0.8420960015261783  0.023573949735077922  1.469676372472043


In [7]:
// Computing Mahalanobis Distance 

val mahalanobis = udf[Double, org.apache.spark.ml.linalg.Vector]{ v =>
 val k = v.toArray
 val vB = new DenseVector(k);
 vB.t * invColCovB * vB
}

val dfMahalanobis = dfScaled.withColumn("mahalanobis", mahalanobis(dfScaled("scaledFeat")))

mahalanobis = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))
dfMahalanobis = [id: bigint, scaledFeat: vector ... 1 more field]


[id: bigint, scaledFeat: vector ... 1 more field]

In [8]:
// Computing Mahalanobis Distance 
dfMahalanobis.show()

+---+--------------------+------------------+
| id|          scaledFeat|       mahalanobis|
+---+--------------------+------------------+
|  0|[-0.4547149615137...| 0.395213334194827|
|  1|[-0.0514304195269...|1.0136570223400254|
|  2|[0.16706755596984...| 3.683608028817502|
|  3|[-0.8604783575193...|2.9395558419372585|
|  4|[-0.4932162279053...|0.3414104330535219|
|  5|[-0.2727855749315...| 2.186702930869009|
|  6|[-0.3479655155863...| 2.979766249119513|
|  7|[-0.7164656378138...| 2.658093077061608|
|  8|[0.23569420610668...|2.9571704842712463|
|  9|[-0.0364814207361...|2.1515995051504926|
| 10|[2.83077635345669...| 8.693223093184995|
+---+--------------------+------------------+



In [9]:
// Removing Outliers 

dfMahalanobis.sort(dfMahalanobis("mahalanobis").desc).show(2)

val ids = dfMahalanobis.select("id","mahalanobis").sort(dfMahalanobis("mahalanobis").desc).drop("mahalanobis").collect() 

val idOutliers = ids.map(_(0).asInstanceOf[Long]).slice(0,2)

+---+--------------------+-----------------+
| id|          scaledFeat|      mahalanobis|
+---+--------------------+-----------------+
| 10|[2.83077635345669...|8.693223093184995|
|  2|[0.16706755596984...|3.683608028817502|
+---+--------------------+-----------------+
only showing top 2 rows



ids = Array([10], [2], [6], [8], [3], [7], [5], [9], [1], [0], [4])
idOutliers = Array(10, 2)


Array(10, 2)

## Removing Outliers

In [10]:
dfOutlier.filter("id not in (10, 2)").show()

+---+--------------------+
| id|            features|
+---+--------------------+
|  0|[0.41371264720975...|
|  1|[0.73117192818966...|
|  3|[0.09430205113458...|
|  4|[0.38340505276222...|
|  5|[0.55692461355235...|
|  6|[0.49774414066138...|
|  7|[0.20766661062014...|
|  8|[0.95719194065089...|
|  9|[0.74293954612044...|
+---+--------------------+



## Lesson Summary

- Having completed this lesson, you should be able to:
-	Compute the inverse of covariance matrix given of a dataset 
-	Compute Mahalanobis Distance for all elements in a dataset
-	Remove outliers from a dataset

### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.